<a href="https://colab.research.google.com/github/Raidepp/CapstoneNom-backend/blob/main/ProjectNom_Iter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!cp ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kmader/food41
!unzip food41.zip

In [ ]:
import h5py
import matplotlib.pyplot as plt

train_h5 = 'food_c101_n10099_r64x64x3.h5'
test_h5 = 'food_test_c101_n1000_r64x64x3.h5'

fl = h5py.File(test_h5, 'r')
test_img = fl['images']

#print(test_img[0])

# Creating DataGenerator

In [11]:
from shutil import copytree, rmtree

# # Try using few labels
# def create_subset_func(ls, dest):
#   if os.path.exists(dest):
#     rmtree(dest)
#   os.makedirs(dest)

#   for item in ls:
#     # Copying all images in picked list into new directory
#     copytree( os.path.join('images', item), os.path.join(dest, item) )

# pick_label = ['donuts', 'carrot_cake', 'tiramisu', 'fried_rice', 'french_fries', 'omelette',
#               'takoyaki', 'hot_dog', 'creme_brulee', 'chocolate_mousse', 'ramen', ]

# pick_label = ['donuts', 'tiramisu', 'creme_brulee', 'chocolate_mousse' ]  # 'carrot_cake', 
# pick_label = ['pizza', 'omelette', 'apple_pie']
# pick_label = ['apple_pie', 'beef_carpaccio', 'bibimbap', 'cup_cakes', 'foie_gras', 
#               'french_fries', 'garlic_bread', 'pizza', 'spring_rolls', 'spaghetti_carbonara',
#               'strawberry_shortcake']
# dest = 'subset_images'
# create_subset_func(pick_label, dest)

# Load the label of model from labels.txt
with open('./meta/meta/labels.txt') as file:
    lines = file.readlines()
    labels = [line.rstrip() for line in lines]

print(labels)
print("Total item: ", len(labels))

['Apple pie', 'Baby back ribs', 'Baklava', 'Beef carpaccio', 'Beef tartare', 'Beet salad', 'Beignets', 'Bibimbap', 'Bread pudding', 'Breakfast burrito', 'Bruschetta', 'Caesar salad', 'Cannoli', 'Caprese salad', 'Carrot cake', 'Ceviche', 'Cheesecake', 'Cheese plate', 'Chicken curry', 'Chicken quesadilla', 'Chicken wings', 'Chocolate cake', 'Chocolate mousse', 'Churros', 'Clam chowder', 'Club sandwich', 'Crab cakes', 'Creme brulee', 'Croque madame', 'Cup cakes', 'Deviled eggs', 'Donuts', 'Dumplings', 'Edamame', 'Eggs benedict', 'Escargots', 'Falafel', 'Filet mignon', 'Fish and chips', 'Foie gras', 'French fries', 'French onion soup', 'French toast', 'Fried calamari', 'Fried rice', 'Frozen yogurt', 'Garlic bread', 'Gnocchi', 'Greek salad', 'Grilled cheese sandwich', 'Grilled salmon', 'Guacamole', 'Gyoza', 'Hamburger', 'Hot and sour soup', 'Hot dog', 'Huevos rancheros', 'Hummus', 'Ice cream', 'Lasagna', 'Lobster bisque', 'Lobster roll sandwich', 'Macaroni and cheese', 'Macarons', 'Miso sou

In [12]:
BATCH_SIZE = 128
IMG_WIDTH, IMG_HEIGHT = 299, 299

def data_generator_func(data_dir, img_width, img_height, batch_size):
    train_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=60,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest',
                                      validation_split=0.2)
    
    train_generator = train_datagen.flow_from_directory(directory=data_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        class_mode='categorical',
                                                        subset='training')
    
    val_datagen = ImageDataGenerator(rescale=1./255,
                                    validation_split=0.2)
    
    val_generator = val_datagen.flow_from_directory(directory=data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='validation')
    
    return train_generator, val_generator

#train_generator, val_generator = data_generator_func('subset_images', IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE)
#train_generator, val_generator = data_generator_func(DATA_DIR, IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE)
train_generator, val_generator = data_generator_func('images', IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE)


Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [13]:
num_train_generator = 80800
num_val_generator = 20200

In [ ]:
labels_dict = train_generator.class_indices
labels_dict

# Continuation of Last Model

## Notes of learning rate

One of hyperparameter to tune.
> The learning rate is perhaps the most important hyperparameter. If you have time to tune only one hyperparameter, tune the learning rate.
 - Page 429, Deep Leanring, 2016

Unfortunately, discovering good learning rate only through trial-and-error.

---
Learning rate interact with other aspect of optimization processs, and interaction may be nonlinear. Generally, smaller learning rate require more training epochs. Further, **smaller batch size better suited for smaller learning rate**, given the noisy estimate of the error gradient.

In [18]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

input_shape = 299
icv3 = InceptionV3(input_shape=(input_shape, input_shape, 3),
                   weights='imagenet',
                   include_top=False)

for layer in icv3.layers:
    layer.trainable = False

## Notes on Batch Size, Step per epochs, and Validation steps

Sc: https://datascience.stackexchange.com/questions/29719/how-to-set-batch-size-steps-per-epoch-and-validation-steps

- batch_size, determine number of samples in each mini batch. 
  - Maximum is all samples, result in gradient descent accurate with loss into minimum (if learning rate small enough), but iterations are slower.
  - Minimum is 1, result in stochastic gradient descent: fast but the direction of gradient step based on only 1 sample. 

- steps_per_epoch, number of batch iterations before training epochs considered finished.

- validation_steps, same as steps_per_epoch but for validation set.

In [ ]:
def train_model(output_units):
  model = tf.keras.Sequential([
      icv3,
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(output_units, kernel_regularizer=tf.keras.regularizers.l2(5e-3), activation='softmax')
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])
  
  # Creating callbacks
  checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='model/checkpoint/checkpoint_model.hdf5',
                                                          save_best_only=True,
                                                          verbose=1)

  csvlogger_callback = tf.keras.callbacks.CSVLogger('model/csv_log/csvlog_model.log')

  # Fit the model
  history = model.fit(
      train_generator,
      validation_data=val_generator,
      batch_size=128,
      steps_per_epoch=num_train_generator // BATCH_SIZE,
      validation_steps=num_val_generator // BATCH_SIZE,
      epochs=30,
      callbacks=[csvlogger_callback, checkpoint_callback],
      verbose=1
  )

  return model, history

In [ ]:
# cont_model_icv3 = tf.keras.Sequential([
#     icv3,
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(3, kernel_regularizer=tf.keras.regularizers.l2(5e-3), activation='softmax')
# ])

# fifth_model_icv3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#                           loss=tf.keras.losses.CategoricalCrossentropy(),
#                           metrics=['accuracy'])

# # Creating callbacks
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='model/checkpoint/checkpoint5th_3class_adam.hdf5',
#                                                          save_best_only=True,
#                                                          verbose=1)

# csvlogger_callback = tf.keras.callbacks.CSVLogger('model/csv_log/csvlog5th_3class_adam.log')

# # Fit the model
# history_fifth_model_icv3 = fifth_model_icv3.fit(
#     train_generator,
#     validation_data=val_generator,
#     batch_size=128,
#     steps_per_epoch=num_train_generator // BATCH_SIZE,
#     validation_steps=num_val_generator // BATCH_SIZE,
#     epochs=30,
#     callbacks=[csvlogger_callback, checkpoint_callback],
#     verbose=1
# )

# fifth_model_icv3.save('model/model5th_icv3_adam.hdf5')

In [ ]:
cont_model, cont_history = train_model(11)

Epoch 1/30
68/68 [==============================] - ETA: 0s - loss: 1.7284 - accuracy: 0.4910
Epoch 1: val_loss improved from inf to 0.93304, saving model to model/checkpoint/checkpoint_model.hdf5
68/68 [==============================] - 173s 2s/step - loss: 1.7284 - accuracy: 0.4910 - val_loss: 0.9330 - val_accuracy: 0.7233
Epoch 2/30
68/68 [==============================] - ETA: 0s - loss: 1.0504 - accuracy: 0.6960
Epoch 2: val_loss improved from 0.93304 to 0.81535, saving model to model/checkpoint/checkpoint_model.hdf5
68/68 [==============================] - 153s 2s/step - loss: 1.0504 - accuracy: 0.6960 - val_loss: 0.8154 - val_accuracy: 0.7679
Epoch 3/30
68/68 [==============================] - ETA: 0s - loss: 0.9297 - accuracy: 0.7294
Epoch 3: val_loss improved from 0.81535 to 0.74579, saving model to model/checkpoint/checkpoint_model.hdf5
68/68 [==============================] - 153s 2s/step - loss: 0.9297 - accuracy: 0.7294 - val_loss: 0.7458 - val_accuracy: 0.7946
Epoch 4/30


In [ ]:
cont_model.save("model/cont_model_1")

!zip -r ./cont_model_1.zip model

from google.colab import files
files.download("./cont_model_1.zip")

INFO:tensorflow:Assets written to: model/cont_model_1/assets
  adding: model/ (stored 0%)
  adding: model/checkpoint/ (stored 0%)
  adding: model/checkpoint/checkpoint_model.hdf5 (deflated 8%)
  adding: model/.ipynb_checkpoints/ (stored 0%)
  adding: model/csv_log/ (stored 0%)
  adding: model/csv_log/csvlog_model.log (deflated 52%)
  adding: model/cont_model_1/ (stored 0%)
  adding: model/cont_model_1/assets/ (stored 0%)
  adding: model/cont_model_1/keras_metadata.pb (deflated 96%)
  adding: model/cont_model_1/saved_model.pb (deflated 92%)
  adding: model/cont_model_1/variables/ (stored 0%)
  adding: model/cont_model_1/variables/variables.index (deflated 76%)
  adding: model/cont_model_1/variables/variables.data-00000-of-00001 (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Training on all classes

In [22]:
# Initializing Tensorboards
import datetime

def tensorboard_callback():
  log_dir = "./drive/MyDrive/Capstone_Related/model_icv3_101class/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")

  return tensorboard_callback

In [23]:
INPUT_SHAPE = (299, 299, 3)
BATCH_SIZE = 128
NUM_CLASSES = 101

icv3 = tf.keras.applications.InceptionV3(input_shape=INPUT_SHAPE, include_top=False)

def create_model(base_model, num_classes=NUM_CLASSES):
  # Freezing EfficientNet layer
  base_model.trainable = False

  x = base_model.output
  x = tf.keras.layers.MaxPooling2D()(x)
  x = tf.keras.layers.Dense(units=512, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Flatten()(x)
  outputs = tf.keras.layers.Dense(units=num_classes, kernel_regularizer=tf.keras.regularizers.l2(5e-3), activation='softmax')(x)

  model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

  # Compiling model
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])
  
  return model

In [24]:
model_icv3 = create_model(icv3)
model_path = "./drive/MyDrive/Capstone_Related/model_icv3_101class"

# Creating callbacks
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='./drive/MyDrive/Capstone_Related/model_icv3_101class/checkpoint/checkpoint_icv3.hdf5',
                                                          save_best_only=True,
                                                          verbose=1)

csvlogger_callback = tf.keras.callbacks.CSVLogger('./drive/MyDrive/Capstone_Related/model_icv3_101class/csv_log/csvlog_icv3.log')

history_icv3 = model_icv3.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    steps_per_epoch=num_train_generator // BATCH_SIZE,
    validation_steps=num_val_generator // BATCH_SIZE,
    callbacks=[csvlogger_callback, checkpoint_callback, tensorboard_callback()],
    verbose=1
)

Saving TensorBoard log files to: ./drive/MyDrive/Capstone_Related/model_icv3_101class/20220605-091022
Epoch 1/30
631/631 [==============================] - ETA: 0s - loss: 3.8861 - accuracy: 0.2891
Epoch 1: val_loss improved from inf to 2.81080, saving model to ./drive/MyDrive/Capstone_Related/model_icv3_101class/checkpoint/checkpoint_icv3.hdf5
631/631 [==============================] - 2017s 3s/step - loss: 3.8861 - accuracy: 0.2891 - val_loss: 2.8108 - val_accuracy: 0.4801
Epoch 2/30
631/631 [==============================] - ETA: 0s - loss: 2.8877 - accuracy: 0.4446
Epoch 2: val_loss improved from 2.81080 to 2.44429, saving model to ./drive/MyDrive/Capstone_Related/model_icv3_101class/checkpoint/checkpoint_icv3.hdf5
631/631 [==============================] - 1986s 3s/step - loss: 2.8877 - accuracy: 0.4446 - val_loss: 2.4443 - val_accuracy: 0.5324
Epoch 3/30
631/631 [==============================] - ETA: 0s - loss: 2.5690 - accuracy: 0.4889
Epoch 3: val_loss improved from 2.44429 to

KeyboardInterrupt: ignored

In [25]:
import pickle

with open('./history_icv3', 'wb') as file_pi:
  pickle.dump(history_icv3.history, file_pi)

NameError: ignored

# First EfficientNetB2 Model

In [ ]:
# Execute this when loading model from previous checkpoint
model_path = "./model/checkpoint/checkpoint_model.hdf5"

model_efnetb2_1 = tf.keras.models.load_model(model_path)

In [8]:
# EfficientNet doesn't use rescale as its already in their preprocessing layer

BATCH_SIZE = 128
# IMG_WIDTH, IMG_HEIGHT = 224, 224  # For EfficientNetB0
IMG_WIDTH, IMG_HEIGHT = 260, 260  # For EfficientNetB2

def data_generator_func(data_dir, img_width, img_height, batch_size):
    train_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=60,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest',
                                      validation_split=0.2)
    
    train_generator = train_datagen.flow_from_directory(directory=data_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        class_mode='categorical',
                                                        subset='training')
    
    val_datagen = ImageDataGenerator(rescale=1./255,
                                    validation_split=0.2)
    
    val_generator = val_datagen.flow_from_directory(directory=data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='validation')
    
    return train_generator, val_generator

#train_generator, val_generator = data_generator_func('subset_images', IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE)
#train_generator, val_generator = data_generator_func(DATA_DIR, IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE)
train_generator, val_generator = data_generator_func('images', IMG_WIDTH, IMG_HEIGHT, BATCH_SIZE)


Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
#from tensorflow.keras.applications import EfficientNetB2

In [ ]:
INPUT_SHAPE = (260, 260, 3)
BATCH_SIZE = 128
efnetb2 = tf.keras.applications.EfficientNetB2(input_shape=INPUT_SHAPE, include_top=False)

def create_model(base_model, num_classes=11):
  # Freezing EfficientNet layer
  base_model.trainable = False

  x = base_model.output
  x = tf.keras.layers.MaxPooling2D()(x)
  x = tf.keras.layers.Dense(units=512, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Flatten()(x)
  outputs = tf.keras.layers.Dense(units=num_classes, kernel_regularizer=tf.keras.regularizers.l2(5e-3), activation='softmax')(x)

  model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

  # Compiling model
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])
  
  return model

In [ ]:
model_efnetb2_1 = create_model(efnetb2)

# Creating callbacks
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='model/checkpoint/checkpoint_model.hdf5',
                                                          save_best_only=True,
                                                          verbose=1)

csvlogger_callback = tf.keras.callbacks.CSVLogger('model/csv_log/csvlog_model.log')

history_efnetb2_1 = model_efnetb2_1.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    steps_per_epoch=num_train_generator // BATCH_SIZE,
    validation_steps=num_val_generator // BATCH_SIZE,
    callbacks=[csvlogger_callback, checkpoint_callback, tensorboard_callback("efnet_ver1")],
    verbose=1
)

Saving TensorBoard log files to: ./model/tfboardefnet_ver1/20220531-095348
Epoch 1/30
68/68 [==============================] - ETA: 0s - loss: 1.6151 - accuracy: 0.5180
Epoch 1: val_loss improved from inf to 0.76246, saving model to model/checkpoint/checkpoint_model.hdf5
68/68 [==============================] - 185s 3s/step - loss: 1.6151 - accuracy: 0.5180 - val_loss: 0.7625 - val_accuracy: 0.7941
Epoch 2/30
68/68 [==============================] - ETA: 0s - loss: 0.8646 - accuracy: 0.7561
Epoch 2: val_loss improved from 0.76246 to 0.60942, saving model to model/checkpoint/checkpoint_model.hdf5
68/68 [==============================] - 177s 3s/step - loss: 0.8646 - accuracy: 0.7561 - val_loss: 0.6094 - val_accuracy: 0.8410
Epoch 3/30
68/68 [==============================] - ETA: 0s - loss: 0.7327 - accuracy: 0.7943
Epoch 3: val_loss improved from 0.60942 to 0.54667, saving model to model/checkpoint/checkpoint_model.hdf5
68/68 [==============================] - 168s 2s/step - loss: 0.73

In [ ]:
!zip -r ./model_efnetb2_1.zip model

from google.colab import files
files.download("./model_efnetb2_1.zip")

  adding: model/ (stored 0%)
  adding: model/checkpoint/ (stored 0%)
  adding: model/checkpoint/checkpoint_model.hdf5 (deflated 10%)
  adding: model/checkpoint/.ipynb_checkpoints/ (stored 0%)
  adding: model/tfboardefnet_ver1/ (stored 0%)
  adding: model/tfboardefnet_ver1/20220531-095348/ (stored 0%)
  adding: model/tfboardefnet_ver1/20220531-095348/validation/ (stored 0%)
  adding: model/tfboardefnet_ver1/20220531-095348/validation/events.out.tfevents.1653990991.92391deb65b5.75.2.v2 (deflated 77%)
  adding: model/tfboardefnet_ver1/20220531-095348/train/ (stored 0%)
  adding: model/tfboardefnet_ver1/20220531-095348/train/events.out.tfevents.1653990830.92391deb65b5.75.1.v2 (deflated 93%)
  adding: model/tfboardefnet_ver1/.ipynb_checkpoints/ (stored 0%)
  adding: model/.ipynb_checkpoints/ (stored 0%)
  adding: model/csv_log/ (stored 0%)
  adding: model/csv_log/.ipynb_checkpoints/ (stored 0%)
  adding: model/csv_log/csvlog_model.log (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# TensorBoard Visualize and Playground

In [ ]:
import datetime

def tensorboard_callback(experiment_name):
  log_dir = "./model/tfboard" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")

  return tensorboard_callback

In [ ]:
train_generator.samples

8800

In [ ]:
# Unzip files
!unzip model_efnet_1.zip

Archive:  model_efnet_1.zip
   creating: model/
   creating: model/checkpoint/
  inflating: model/checkpoint/checkpoint_model.hdf5  
   creating: model/tfboardefnet_ver1/
   creating: model/tfboardefnet_ver1/20220529-132959/
   creating: model/tfboardefnet_ver1/20220529-132959/train/
  inflating: model/tfboardefnet_ver1/20220529-132959/train/events.out.tfevents.1653831002.3fcddb13b51f.85.0.v2  
   creating: model/tfboardefnet_ver1/20220529-133135/
   creating: model/tfboardefnet_ver1/20220529-133135/validation/
  inflating: model/tfboardefnet_ver1/20220529-133135/validation/events.out.tfevents.1653831273.3fcddb13b51f.85.2.v2  
   creating: model/tfboardefnet_ver1/20220529-133135/train/
  inflating: model/tfboardefnet_ver1/20220529-133135/train/events.out.tfevents.1653831097.3fcddb13b51f.85.1.v2  
   creating: model/.ipynb_checkpoints/
   creating: model/csv_log/
  inflating: model/csv_log/csvlog_model.log  
